<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

The nonlinear system
$$ \begin{align*}
3x_1 - \cos(x_2 x_3)-\frac{1}{2} &= 0 \\
x_1^2 - 625 x_2^2 - \frac{1}{4} &= 0 \\
\exp^{-x_1 x_2}+20x_3+\frac{10\pi-3}{3} &= 0
\end{align*}
$$
has a singular Jacobian matrix at the solution. Apply Newton’s method with $x^{(0)} = (1, 1 − 1)^t$. Note that convergence may be slow or may not occur within a reasonable number of iterations.

In [1]:
import numpy as np
from numpy import linalg
from abc import abstractmethod
import pandas as pd
import math

pd.options.display.float_format = '{:,.8f}'.format
np.set_printoptions(suppress=True, precision=8)

TOR = pow(10.0, -9)
MAX_ITR = 150

In [2]:
class NewtonMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

In [3]:
class Newton(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 3 * x[0] - math.cos(x[1] * x[2]) - 1 / 2
        sol[1] = pow(x[0], 2) - 625 * pow(x[1], 2) - 1 / 4
        sol[2] = math.exp(-x[0] * x[1]) + 20 * x[2] + (10 * math.pi - 3) / 3
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(3, 3))
        jac[0][0] = 3
        jac[0][1] = x[2] * math.sin(x[1] * x[2])
        jac[0][2] = x[1] * math.sin(x[1] * x[2])
        jac[1][0] = 2 * x[0]
        jac[1][1] = -1250 * x[1]
        jac[1][2] = 0
        jac[2][0] = -x[1] * math.exp(-x[0] * x[1])
        jac[2][1] = -x[0] * math.exp(-x[0] * x[1])
        jac[2][2] = 20
        return jac

    def run(self, x):
        """
        given x_0 in R^3 as a starting point.

        :param x: x_0 as described
        :return: the minimizer x* of f
        """
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]

        for k in range(MAX_ITR):
            jac = self.jacobian(x)
            f = -self.f(x)
            y = linalg.solve(jac, f)
            nx = x + y
            residual = linalg.norm(x - nx, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][4] = linalg.norm(x - xk, np.inf)

        print(self.f(x))
        return df

In [4]:
x0 = np.array([1, 1, -1])
Newton().run(x0)

[ 0. -0.  0.]


,x1,x2,x3,residual,actual-residual
0,1.00000000,1.00000000,-1.00000000,nan,1.00000000
1,0.62498925,0.49999998,-0.50808767,0.50000002,0.49999998
2,0.49952031,0.24997404,-0.51819092,0.25002595,0.24997404
3,0.49997146,0.12498693,-0.52047975,0.12498711,0.12498693
4,0.49999553,0.06249343,-0.52203737,0.06249349,0.06249343
5,0.49999945,0.03124671,-0.52281773,0.03124672,0.03124671
6,0.49999993,0.01562335,-0.52320821,0.01562336,0.01562335
7,0.49999999,0.00781168,-0.52340349,0.00781168,0.00781168
8,0.50000000,0.00390584,-0.52350113,0.00390584,0.00390584
9,0.50000000,0.00195292,-0.52354995,0.00195292,0.00195292


In [5]:
x0 = np.array([5, 5, -0.5])
Newton().run(x0)

[ 0. -0.  0.]


,x1,x2,x3,residual,actual-residual
0,5.00000000,5.00000000,-0.50000000,nan,5.00000000
1,0.17569112,2.49624111,-0.47359878,4.82430888,2.49624111
2,0.46283157,1.24808266,-0.48980399,1.24815845,1.24808266
3,0.49359985,0.62403665,-0.50868666,0.62404601,0.62403665
4,0.49939356,0.31201750,-0.51586997,0.31201914,0.31201750
5,0.49992668,0.15600856,-0.51971077,0.15600894,0.15600856
6,0.49999123,0.07800424,-0.52165039,0.07800433,0.07800424
7,0.49999893,0.03900211,-0.52262395,0.03900213,0.03900211
8,0.49999987,0.01950105,-0.52311128,0.01950106,0.01950105
9,0.49999998,0.00975052,-0.52335502,0.00975053,0.00975052
